In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [3]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True, 
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_datloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [6]:
loss_fn = nn.CrossEntropyLoss()

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss: >7f} [{current: >5d}/{size: >5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
for t in range(epochs):
    print(f"Epoch {t + 1} \n -----------------------------------")
    train_loop(train_datloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1 
 -----------------------------------
loss: 2.324793 [   64/60000]
loss: 2.305337 [ 6464/60000]
loss: 2.289733 [12864/60000]
loss: 2.273533 [19264/60000]
loss: 2.258209 [25664/60000]
loss: 2.245527 [32064/60000]
loss: 2.240017 [38464/60000]
loss: 2.215250 [44864/60000]
loss: 2.206655 [51264/60000]
loss: 2.178045 [57664/60000]
Test Error: 
 Accuracy: 45.6%, Avg loss: 2.169678 

Epoch 2 
 -----------------------------------
loss: 2.185144 [   64/60000]
loss: 2.169362 [ 6464/60000]
loss: 2.119123 [12864/60000]
loss: 2.131674 [19264/60000]
loss: 2.076854 [25664/60000]
loss: 2.032559 [32064/60000]
loss: 2.050128 [38464/60000]
loss: 1.976468 [44864/60000]
loss: 1.983991 [51264/60000]
loss: 1.914548 [57664/60000]
Test Error: 
 Accuracy: 57.8%, Avg loss: 1.907894 

Epoch 3 
 -----------------------------------
loss: 1.942355 [   64/60000]
loss: 1.909748 [ 6464/60000]
loss: 1.796332 [12864/60000]
loss: 1.836456 [19264/60000]
loss: 1.720767 [25664/60000]
loss: 1.677150 [32064/60000]
loss